# FIT5196 Assessment 1 - EDA

Due Date: 23:55, Sunday, 14 September 2025


---



#### Group 35:
Member 1: Adrian Leong Tat Wei, (27030768), atleo4@student.monash.edu, Contribution

Member 2: Full Name, (Student ID), Email, Contribution

...

---

### Table of Content

1.
2.
3.



## 1. Load, parse and merge data files






In [ ]:
# Alternatives: BeautifulSoup, lxml 
import xml.etree.ElementTree as ET

# Passing the path of the
# xml document to enable the
# parsing process
tree = ET.parse('Group035.xml')

# getting the parent tag of
# the xml document
root = tree.getroot()

# printing the root (parent) tag
# of the xml document, along with
# its memory location
print(root)

# printing the attributes of the
# first tag from the parent 
print(root[0].attrib)
print(root[0].text)

# printing the text contained within
# first subtag of the 5th tag from
# the parent
print(root[0][0].attrib)
print(root[0][0].text)


In [ ]:
# https://www.geeksforgeeks.org/python/read-json-file-using-python/
import json

# Open and read the JSON file
with open('Group035.json', 'r') as file:
    data = json.load(file)
    print(data)
    # # Iterating through the json list
    # for i in data['PostID']:
    #     print(i)

# Closing file
file.close()


### 1.1 Load data files

### 1.2 Merge dataset

## 2. EDA

### 2.1 Dataset overview

### 2.2 Univariate analysis

### 2.3 Bivariate analysis

### 2.4 Multivariate analysis

## 3. Key insights and research questions

### 3.1 Key findings

### 3.2 Machine Learning research questions and justification

## Reference